In [18]:
import nltk
import numpy
import json
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import sklearn
import string
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
lemmatizer = WordNetLemmatizer()

#Current Classifier: 1 is Greeting, 2 is Thank you, 3 is Product information

df = pd.read_csv("intentdata.csv")
df

,Pattern,Greetings,Thanks,Product,Shipping Times,Payments,Refund
0,Hello!,1,0,0,0,0,0
1,Hi there!,1,0,0,0,0,0
2,Good morning!,1,0,0,0,0,0
3,Good afternoon!,1,0,0,0,0,0
4,Good evening!,1,0,0,0,0,0
...,...,...,...,...,...,...,...
141,Can I receive a refund for a digital product o...,0,0,0,0,0,1
142,What if the item I want to return was part of ...,0,0,0,0,0,1
143,Do you offer refunds on sale items or clearanc...,0,0,0,0,0,1
144,Can I return an item to a physical store locat...,0,0,0,0,0,1


In [19]:
# Used from the d2l.
def clean(doc): # doc is a string of text
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([lemmatizer.lemmatize(word.lower()) for word in doc.split()])
    # remove punctuation and numbers
    return doc

In [20]:
#Preprocess the pandas dataframe
df['Pattern'] = df["Pattern"].apply(clean)

In [21]:
vect = CountVectorizer()
vect.fit(df["Pattern"])

X = vect.transform(df["Pattern"])
dfnew = pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out())
dfnew['Greetings'] = df['Greetings']
dfnew['Thanks'] = df['Thanks']
dfnew['Product'] = df['Product']
dfnew['Shipping Times'] = df['Shipping Times']
dfnew['Payments'] = df['Payments']
dfnew['Refund'] = df['Refund']

In [22]:
print(sorted(vect.vocabulary_))
with open('vocabulary.txt', 'w') as filehandle:
    json.dump(sorted(vect.vocabulary_), filehandle)

['about', 'accept', 'accepted', 'additional', 'after', 'afternoon', 'ahoy', 'amazing', 'an', 'and', 'anticipate', 'anticipated', 'any', 'apple', 'appreciate', 'appreciative', 'are', 'area', 'arrival', 'arrive', 'associated', 'at', 'available', 'awesome', 'back', 'bank', 'be', 'been', 'best', 'bottom', 'bought', 'bundle', 'buy', 'by', 'can', 'cancel', 'cannot', 'cant', 'card', 'carry', 'category', 'certain', 'check', 'clearance', 'company', 'complete', 'confirmation', 'cost', 'couldnt', 'credit', 'currency', 'damaged', 'date', 'day', 'deal', 'debit', 'debt', 'deeply', 'defective', 'delivered', 'delivery', 'detail', 'different', 'digital', 'dispatched', 'do', 'doe', 'done', 'dont', 'doorstep', 'drop', 'duration', 'enough', 'estimate', 'estimated', 'evening', 'everything', 'example', 'exchange', 'expect', 'expected', 'expedited', 'fastest', 'fee', 'for', 'form', 'from', 'full', 'gateway', 'get', 'gift', 'give', 'going', 'good', 'google', 'grateful', 'gratitude', 'greeting', 'hanging', 'ha

In [23]:
dfnew

,about,accept,accepted,additional,after,afternoon,ahoy,amazing,an,and,...,you,your,youre,youve,Greetings,Thanks,Product,Shipping Times,Payments,Refund
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
143,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
144,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
X = dfnew.drop('Greetings',axis=1)
X = X.drop('Thanks',axis=1)
X = X.drop('Product',axis=1)
X = X.drop('Shipping Times',axis=1)
X = X.drop('Payments',axis=1)
X = X.drop('Refund',axis=1)
y = dfnew[['Greetings','Thanks','Product', 'Shipping Times', 'Payments', 'Refund']]
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size = 0.2)

In [25]:
X

,about,accept,accepted,additional,after,afternoon,ahoy,amazing,an,and,...,with,without,word,work,wrong,yo,you,your,youre,youve
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
144,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
y

,Greetings,Thanks,Product,Shipping Times,Payments,Refund
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
141,0,0,0,0,0,1
142,0,0,0,0,0,1
143,0,0,0,0,0,1
144,0,0,0,0,0,1


In [27]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [28]:
X_train.shape

(116, 271)

In [29]:
y_train.shape

(116, 6)

In [30]:
"""
print("--Make model--")
model = Sequential()
model.add(Dense(32,input_shape=(85,),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense())
"""
print("--Make model--")
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

--Make model--
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                8704      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 390       
                                                                 
Total params: 11,206
Trainable params: 11,206
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
print("--Fit model--")
model.fit(X_train, y_train, epochs=100)

--Fit model--
Epoch 1/100
4/4 [==============================] - 2s 6ms/step - loss: 1.7857 - accuracy: 0.1724
Epoch 2/100
4/4 [==============================] - 0s 10ms/step - loss: 1.7812 - accuracy: 0.2155
Epoch 3/100
4/4 [==============================] - 0s 10ms/step - loss: 1.7807 - accuracy: 0.1983
Epoch 4/100
4/4 [==============================] - 0s 7ms/step - loss: 1.7644 - accuracy: 0.2241
Epoch 5/100
4/4 [==============================] - 0s 8ms/step - loss: 1.7443 - accuracy: 0.2155
Epoch 6/100
4/4 [==============================] - 0s 8ms/step - loss: 1.7371 - accuracy: 0.2672
Epoch 7/100
4/4 [==============================] - 0s 8ms/step - loss: 1.7113 - accuracy: 0.2414
Epoch 8/100
4/4 [==============================] - 0s 7ms/step - loss: 1.6896 - accuracy: 0.3362
Epoch 9/100
4/4 [==============================] - 0s 6ms/step - loss: 1.6858 - accuracy: 0.3103
Epoch 10/100
4/4 [==============================] - 0s 5ms/step - loss: 1.6673 - accuracy: 0.3103
Epoch 11/100


4/4 [==============================] - 0s 4ms/step - loss: 0.2953 - accuracy: 0.9397
Epoch 85/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2285 - accuracy: 0.9655
Epoch 86/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2514 - accuracy: 0.9483
Epoch 87/100
4/4 [==============================] - 0s 5ms/step - loss: 0.2270 - accuracy: 0.9397
Epoch 88/100
4/4 [==============================] - 0s 5ms/step - loss: 0.2974 - accuracy: 0.9224
Epoch 89/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2439 - accuracy: 0.9224
Epoch 90/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2530 - accuracy: 0.9138
Epoch 91/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2272 - accuracy: 0.9310
Epoch 92/100
4/4 [==============================] - 0s 4ms/step - loss: 0.1927 - accuracy: 0.9741
Epoch 93/100
4/4 [==============================] - 0s 4ms/step - loss: 0.2700 - accuracy: 0.9310
Epoch 94/100
4/4 [===============

In [32]:
print("--Evaluate model--")
model_loss1, model_acc1 = model.evaluate(X_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(X_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

--Evaluate model--
4/4 - 0s - loss: 0.0314 - accuracy: 1.0000 - 297ms/epoch - 74ms/step
1/1 - 0s - loss: 0.2729 - accuracy: 0.9000 - 52ms/epoch - 52ms/step
Train / Test Accuracy: 100.0% / 90.0%


In [33]:
model.save('chatbot_WIP.h5')